## Project Expat Assistance

Starting with the Web Framework with Flask

In [ ]:
#installing the flask

#remove in case flask is not installed
#pip uninstall flask 

#in case upgrade needed
#!pip install --upgrade flask

#flask --version


###############IMPORTANT##########
#Need to add functionality to check if users cannot login with urls

## Running Example code with Flask

In [ ]:
from flask import Flask, render_template, redirect, url_for, request  #importing libraries
import sqlite3
import os


app = Flask(__name__)    #__name__ is communicating with Flask function for user's
                         #current directory specification


#routing in FLask
@app.route("/")          #Calling flask to route to the end of a website eg"xcvb.com/" to execute a code
def Landing_Page():         #function Landing Page at/
    try:
        return render_template("home.html")
    except Exception as e:
        return str(e)            #Just showing the error if any


def get_db_connection():
    conn = sqlite3.connect('expat_project.db')
    conn.row_factory = sqlite3.Row
    return conn

@app.route("/login", methods=['POST','GET'])
def login():
    if request.method == 'POST':
        
        connection = get_db_connection()
        cursor = connection.cursor()
        
        email = request.form['email']
        password = request.form['psw']
        
        cursor.execute('SELECT id, email, pass_hash FROM users where email=? AND pass_hash=?', (email, password))
        
        results = cursor.fetchall()
        
        if len(results) == 0:
            return render_template("LoginError.html")
            
        else:
            user = results[0]
            print("got back user: {}".format(user))
            return render_template("Logged.html", user = user)
    
    


@app.route("/registered", methods=['POST','GET'])
def registered():
#    con=sqlite3.connect('expat_project.db')
    if request.method == 'POST':
        try:
            first_name = request.form['name']
            last_name = request.form['surname']
            email = request.form['email']
            pass_hash = request.form['psw']
            
            with sqlite3.connect('expat_project.db') as con:
                c=con.cursor()
                c.execute("SELECT * FROM users WHERE email =?",(email,))

                check = c.fetchone()
                
                c.execute("INSERT INTO users (first_name,last_name,email,pass_hash) VALUES (?,?,?,?)",(first_name,last_name,email,pass_hash))
                    
                con.commit()
                msg = "Welcome to Expat Assistance "+first_name
                redirect("/registered")

#When the page is refreshed, submit button does not work
        except Exception as e:
            con.rollback()
            msg = "Got back error to register " + str(e)
            
        finally:
            con.close()
            return render_template("Registered.html", msg=msg)
            
            
            

            
########################################################################################

    
#page to select a new application or see existing, need to add buttons here to see existing apps or start a new one 
@app.route("/welcome")
def welcome():
    return render_template('welcome.html')


#page to select application type


@app.route("/users/<user_id>/select_application")
def select_application(user_id):
    conn = get_db_connection()
    rows = conn.execute("SELECT id, title FROM application_types").fetchall()
    conn.close()
    return render_template('select_application_type.html', rows = rows, user_id = user_id) 

#page and query to get existing cases for a user 

@app.route("/users/<user_id>/cases")
def user_cases(user_id):
    conn = get_db_connection()
    rows = conn.execute("""SELECT
        t.title,
        u.first_name as "user_first_name",
        u.last_name as "user_last_name",
        a.created_at,
        a.updated_at,
        c.first_name as "consultant_first_name",
        c.last_name as "consultant_last_name",
        c.company_name,
        a.status 
    FROM applications AS a
    LEFT JOIN application_types as t 
    ON a.application_type_id = t.id
    LEFT JOIN users AS u
    ON a.creator_id = u.id
    LEFT JOIN consultants AS c
    ON a.assigned_consultant_id = c.id
    WHERE a.creator_id = """ + user_id).fetchall()
    conn.close()
    return render_template('user_cases.html', rows = rows) 

#page to start a new application

@app.route("/users/<user_id>/applications/<application_type_id>/create")
def start_application(user_id, application_type_id):
    conn = get_db_connection()
    rows = conn.execute('SELECT title, id, required FROM questions WHERE application_type_id = ' + application_type_id ).fetchall()
    conn.close()
    return render_template('start_application.html', questions = rows, user_id = user_id, application_type_id = application_type_id)  



@app.route("/users/<user_id>/applications/<application_type_id>/submit",  methods=['POST'])
def submit_application(user_id, application_type_id):
    response = request.form
    application_id = None
    msg = ''
    try:
        con = sqlite3.connect('expat_project.db')
        c = con.cursor()
        application_id = c.execute("INSERT INTO applications (application_type_id, creator_id) VALUES (?,?)",(application_type_id,user_id)).lastrowid
        print("inserted application with id: {}".format(application_id))
        for question_id in response:
            application_response_id = c.execute("INSERT INTO application_responses  (question_id, application_id, response) VALUES (?, ?, ?)", (question_id, application_id, response[question_id])).lastrowid
            print("inserted application response with id: {}".format(application_response_id))
        con.commit()
        msg = "Successfully submitted application with responses"
        
    except Exception as e:
        print("rollbacking the transaction {}!!".format(e))
        msg = "Incorrect submission. {}".format(e)
        con.rollback()
        
    finally:
        con.close()
        return render_template('submit_application.html', response = response, application_id = application_id, msg = msg, user_id = user_id)  
#page for consulatnts to 

@app.route("/consultants/<consultant_id>/cases")
def consultant_cases(consultant_id):
    conn = get_db_connection()
    rows = conn.execute("""SELECT
     a.id, 
     t.title,
     u.first_name AS "user_first_name",
     u.last_name AS "user_last_name",
     a.created_at,
     a.updated_at, 
     c.first_name AS "consultant_first_name", 
     c.last_name AS "consultant_last_name",
     c.company_name, 
     a.status
FROM applications as a
LEFT JOIN application_types as t
ON a.application_type_id = t.id
LEFT JOIN users as u
ON 	a.creator_id = u.id
LEFT JOIN consultants as c
ON 	a.assigned_consultant_id = c.id 
WHERE a.assigned_consultant_id =  """ + consultant_id ).fetchall()
    conn.close()
    return render_template('consultant_cases.html', rows = rows) 



app.run(host='localhost', port=8081, debug=True, use_reloader=False) #running app which is Flask at localhost at port 8081





 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://localhost:8081/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jan/2023 16:56:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 16:56:49] "POST /login HTTP/1.1" 200 -


got back user: <sqlite3.Row object at 0x7fcdaa113a90>


127.0.0.1 - - [14/Jan/2023 16:57:22] "GET /users/%3Cuser_id%3E/select_application HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 16:57:24] "GET /users/%3Cuser_id%3E/applications/1/create HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 16:57:28] "GET /users/%3Cuser_id%3E/applications/2/create HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 16:57:30] "GET /users/%3Cuser_id%3E/applications/3/create HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 16:57:34] "GET /users/%3Cuser_id%3E/applications/2/create HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 16:57:36] "GET /users/%3Cuser_id%3E/applications/1/create HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 16:58:00] "GET /users/1/select_application HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 16:58:03] "GET /users/1/applications/2/create HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 16:58:09] "GET /users/1/applications/1/create HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 17:16:03] "POST /login HTTP/1.1" 200 -


got back user: <sqlite3.Row object at 0x7fcdaa113d30>


127.0.0.1 - - [14/Jan/2023 17:17:07] "GET /users/%3Cuser_id%3E/select_application HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 17:17:10] "POST /login HTTP/1.1" 200 -


got back user: <sqlite3.Row object at 0x7fcdaa113fd0>


127.0.0.1 - - [14/Jan/2023 17:17:12] "GET /users/1/select_application HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 17:17:14] "GET /users/1/applications/1/create HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 17:17:16] "POST /users/1/applications/1/submit HTTP/1.1" 200 -


inserted application with id: 19
inserted application response with id: 61
inserted application response with id: 62
inserted application response with id: 63
inserted application response with id: 64
inserted application response with id: 65
inserted application response with id: 66
inserted application response with id: 67
inserted application response with id: 68
inserted application response with id: 69
inserted application response with id: 70
inserted application response with id: 71
inserted application response with id: 72


127.0.0.1 - - [14/Jan/2023 17:17:19] "GET /users/1/cases HTTP/1.1" 200 -
